# 知识图谱与问题翻译

### 概要/说明：
- `include("translatedata.jl")` 导入翻译后的数据
- 翻译文件放在 `translate/`
- 函数工具 `../src/xlsx.jl` 和 `../src/translate.jl`

细节：
- 准备工作
    - 尝试了 Tensor2Tensor 的预训练模型，但官方文档只找到 `德文=>英文` 现成的工具（弃用）
    - 谷歌翻译支持 API 调用，但官方 API 需付费，而 GitHub 第三方编写的 API 不稳定（弃用）
    - 最后使用谷歌提供的文档翻译功能（免费，仅支持 pdf, excel 等，不支持 txt 文本）
    - 决定方案： Excel 进行数据交互，函数工具 `../src/xlsx.jl`
- 翻译训练集和验证集
    - 我们将问题分 `英法` 和 `汉语` 两类进行单语言翻译
    - 翻译前将实体名替换为 "NER" 

> 由于官方提供的 `ILLs(zh-en).txt` 不足以提供所有实体的对齐，三元组翻译放在后边进行

### 导入数据和工具

In [46]:
include("../src/CCKS-mKGQA.jl")
include("../src/tools/xlsx.jl")
include("../src/loaddata/extractdata.jl")
include("../src/process/translate.jl")

remove_discript (generic function with 1 method)

### 翻译三元组

#### 翻译关系

In [47]:
# 获取关系
rels = unique(getindex.(triples, 2))
rep_rels = replace.(rels, rel_rules...)
# 写入 Excel
# write_xlsx("excel_data/triples/rels.xlsx", rep_rels)
MT_rels = strip.(read_xlsx("../old_data/excel_data/triples/rels_MT.xlsx"))
MT_rels = standardrel.(MT_rels) # 去除杂质，转化标准格式
dict_rels = Dict((=>).(rels, MT_rels));

In [11]:
open("translate/rels_raw2new.txt", "w") do io
    for (rel1, rel2) in zip(rels, MT_rels)
        println(io, rel1, '\t', rel2)
    end
end

#### 翻译实体

In [48]:
# 翻译英文图谱中的非英语实体
en_nonen = filter(!isenglish, en_words)
# write_xlsx("excel_data/triples/en_words/en_nonenglish.xlsx", _2space.(en_nonen))
MT_en_nonen = strip.(read_xlsx("../old_data/excel_data/triples/en_words/en_nonenglish_mt.xlsx"))
MT_en_nonen = replace.(MT_en_nonen, ' ' => '_')
en_words_dict = Dict((=>).(en_nonen, MT_en_nonen))

# 翻译中文图谱中不落在英文实体的部分
zh_nonen = filter(∉(Set(en_words)), zh_words)
# write_xlsx("excel_data/triples/zh_words/zh_nonenglish.xlsx", _2space.(zh_nonen))
MT_zh_nonen = strip.(read_xlsx("../old_data/excel_data/triples/zh_words/zh_nonenglish_mt.xlsx"))
MT_zh_nonen = replace.(MT_zh_nonen, ' ' => '_')
zh_words_dict = Dict((=>).(zh_nonen, MT_zh_nonen))

# 翻译整合
dict_words = Dict(union(en_words_dict, zh_words_dict))
dict_words = Dict(word => get(dict_words, word, word) for word in words);

In [8]:
# 写入实体信息
word_io = open("translate/words_raw2new.txt", "w")
for word in words
    println(word_io, word, '\t', dict_words[word])
end
close(word_io)

#### 对齐文件

In [49]:
# 对齐实体：有 3k+ 在翻译后被对齐
MT_ILLs = [(dict_words[en], dict_words[zh]) for (en, zh) in ILLs]
MT_ILLs = filter(i->i[1]!=i[2], MT_ILLs);
length.((ILLs, MT_ILLs))

(13055, 9802)

In [7]:
# open("translate/ILLs(zh-en).txt", "w") do io
#     for (en, zh) in MT_ILLs
#         println(io, en, '\t', zh)
#     end
# end 

#### 三元组 

In [50]:
# 翻译三元组
mt_triple(triple) = mt_triple(triple...)
mt_triple(sub, rel, obj) = (dict_words[sub], dict_rels[rel], dict_words[obj])

mt_triple (generic function with 2 methods)

In [11]:
open("translate/triples.txt", "w") do io
    for triple in unique!(mt_triple.(triples))
        println(io, join(triple, '\t'))
    end
end

### 训练集翻译

In [12]:
# 提取关系
sol2rel(sol) = getindex.(Ref(dict_rels), getindex.(sol, 3))

# 替换实体名
train_ques = [replace(que, _2space(ner) => "NER", '？'=>'?') for (que, ner) in train_ques_ner]
train_sols_mt = [mt_triple.([sol[2:end] for sol in sols]) for sols in train_sols]

# 保存问题
# MT_questions(train_ques, "excel_data/ques/train_ques")

# 读取翻译问题
MT_train_ques = MT_questions(train_ques, "excel_data/ques/train_ques";write=false);

In [13]:
# 写入文件
open("translate/train_ques.txt", "w") do io
    for (que, sols) in zip(MT_train_ques, train_sols_mt)
        ner, rels = sols[1][1], getindex.(sols, 2)
        que = remove_discript(que)
        println(io, que, '\t', ner, '\t', join(rels, '\t'))
    end
end

# open("translate/train_data.txt", "w") do io
#     for (que, sols) in zip(MT_train_ques, train_sols_mt)
#         ner = sols[1][1]
#         println(io, que, '\t', ner)
#         for sol in sols
#             println(io, join(sol, '\t'))
#         end
#         println(io)
#     end
# end

### 验证集翻译

In [57]:
# 读取数据
txts = split(strip(read(open("extract/valid_data_ner.txt", "r"), String)), '\n')
valid_ques_ner = [split(txt, '\t') for txt in txts]

# 关键词替换
valid_ques = [replace(que, _2space(ner) => "NER", '？'=>'?') for (que, ner) in valid_ques_ner]
valid_ners = getindex.(Ref(dict_words), last.(valid_ques_ner));

In [55]:
# 写入文件
MT_questions(valid_ques, "excel_data/ques/valid_ques")

# 读取翻译文件
# valid_ques_MT = MT_questions(valid_ques, "excel_data/ques/valid_ques"; write=false);

In [16]:
# 翻译命名实体-旧命名实体从 zh_triples, en_triples, ILLs 中匹配
open("translate/valid_ques.txt", "w") do io
    for (que, ner) in zip(valid_ques_MT, valid_ners)
        que = remove_discript(que)
        println(io, que, '\t', ner)
    end
end

### 验证集翻译-最终提交

In [58]:
# 读取数据
txts = split(strip(read(open("extract/valid_data_ner.txt", "r"), String)), '\n')
valid_ques_ner = [split(txt, '\t') for txt in txts]
valid_ques = replace.(String.(first.(valid_ques_ner)), '？'=>'?')
valid_ners = getindex.(Ref(dict_words), last.(valid_ques_ner))
# 写入文件
MT_questions(valid_ques, "excel_data/ques/valid_ques")

500×1 Matrix{String}:
 "NER的头衔的作词者的职业是什么"
 "和NER名称一样的人或事物的前任是什么"
 "与1948年战争期间，一艘埃及-英国制造的单桅帆船沉没NER有相同名称的人、事、物的配偶是哪位"
 "NER的选举获胜者的前一任是谁"
 "NER的南部城市属于哪个时区"
 "美国犹他州塞维尔县的一个城镇NER西北部的地点西南部的地点的所属区域是什么"
 "1998年的单曲，由沙奎尔·奥尼尔、dj奎克、塔里克勋爵和彼得·冈兹演唱NER的创作者的继任者是哪位"
 "NER的选举获胜者为哪个总统效力"
 "NER影响到的国家和地区是谁领导的"
 "NER，博茨瓦纳的一个柏格沃克人，它的所在省份的省会是哪里"
 "来自阿拉巴马州的美国政治家NER的头衔属于哪里用什么时区"
 "NER的同名的标志性地点是哪里"
 "NER的继任者的头衔的副手被称为什么"
 ⋮
 "NER的同名是谁的上一任"
 "参与维基媒体列表文章NER的所在城市的命名来源的国家有哪些"
 "和NER，一艘历史悠久的法国船只，它名称一样的人或事物影响了谁"
 "2003年世界摔跤娱乐付费观看活动NER的赞助商属于哪个系列"
 "NER相关的事件由哪位将领领导"
 "NER的同名属于什么分类等级"
 "NER的父亲的出生地是哪里"
 "NER的位置东北部的地点的区划是什么"
 "NER，一项国际足球比赛，其位置西部的地点的区划是什么"
 "NER名称的来源属于哪个领域的"
 "NER的举办地的继任者的继任者是哪位"
 "NER的发源地的立法机构属于哪个党派"

In [59]:
# --经过谷歌翻译--
# 读取翻译文件
valid_ques_MT = MT_questions(valid_ques, "excel_data/ques/valid_ques"; write=false)

# 写入文件
open("translate/valid_ques.txt", "w") do io
    for (que, ner) in zip(valid_ques_MT, valid_ners)
        que = strip(replace.(remove_discript(que), "do you know" => ""))
        println(io, que, '\t', ner)
    end
end

In [60]:
valid_ques

1500-element Vector{String}:
 "who is influenced by the origin of NER's name?"
 "Savez-vous quel est le nom de l" ⋯ 79 bytes ⋯ "t-ball universitaire américain?"
 "NER的头衔的作词者的职业是什么"
 "Savez-vous où est le berceau de la ville de Capitaine de Le club de NER"
 "Qui est le prédécesseurs de Élection après NER"
 "who is the predecessor of the one that is after election of NER, a wahl?"
 "what is the job of the author of NER, an audio drama written by paul magrs?"
 "和NER名称一样的人或事物的前任是什么"
 "Savez-vous où se trouve l'adres" ⋯ 26 bytes ⋯ " stade où s'est déroulée la NER"
 "Savez-vous quelles sont les positions importantes qu'occupe Élection après NER"
 "what is the office of the one that is after election of NER?"
 "À quel parti appartient Élection après NER"
 "who is the predecessor of the p" ⋯ 20 bytes ⋯ "ings with the same name as NER?"
 ⋮
 "NER名称的来源属于哪个领域的"
 "Savez-vous où se déroule la Le nom du navire de NER?"
 "who is earlier than the one that is after election of NER?"
 "Savez-vous où se d